# Part 3

https://health.data.ny.gov/Health/New-York-State-Population-Data-Beginning-2003/e9uj-s3sf <- Data from here

First we will read in the data - initially I chose specific columns to be read in only, but it turns out that there are fields in various columns that contain the "Total" population... so we need to keep the unwanted columns initially to drop these rows. Otherwise, the population data will be the sum of the total population **plus** the sum of the population. Unfortunately, I do not want to simply divide by two in case there is more to it than I am aware of. It just takes a little bit of extra effort. We will be starting with the 2017-2018 season because it had the closest to "normal" distribution. This way I can do some statistical analysis on this subset, and if there is more time I will continue onto other seasons.

In [1]:
import pandas as pd
import numpy as np

In [2]:
path = "../Datasets/"
data = pd.read_csv(path+"NYS_GenderData.csv")
data.head()

,Year,Age Group Code,Age Group Description,Gender Code,Gender Description,Race Ethnicity Code,Race/Ethnicity Description,County Code,County Name,Population
0,2003,0,Total,0,Total,0,Total,1,New York State,19190115
1,2003,0,Total,0,Total,1,White Non Hispanic,1,New York State,11696047
2,2003,0,Total,0,Total,2,Black Non Hispanic,1,New York State,3024588
3,2003,0,Total,0,Total,3,Other Non Hispanic,1,New York State,1337294
4,2003,0,Total,0,Total,5,Hispanic,1,New York State,3132186


As you can see above, the head of the table shows exactly what I am talking about! We have these `Total` rows in many columns: `Age Group Description`, `Gender Description`, and `Race/Ethnicity Description`. First we will make our 2017 subset. If I end up going after more seasons, I will likely remove the rows containing `Total` before making the subsets.

In [3]:
data_17 = data.loc[(data['Year']==2017)]
data_17 = data_17.sort_values(by=['County Name']).reset_index(drop=True)
data_17

,Year,Age Group Code,Age Group Description,Gender Code,Gender Description,Race Ethnicity Code,Race/Ethnicity Description,County Code,County Name,Population
0,2017,9,65 to 74,2,Female,3,Other Non Hispanic,9,Albany,491
1,2017,7,45 to 54,1,Male,1,White Non Hispanic,9,Albany,14526
2,2017,6,35 to 44,2,Female,1,White Non Hispanic,9,Albany,12018
3,2017,6,35 to 44,1,Male,1,White Non Hispanic,9,Albany,12066
4,2017,5,25 to 34,2,Female,1,White Non Hispanic,9,Albany,13525
...,...,...,...,...,...,...,...,...,...,...
11695,2017,7,45 to 54,1,Male,1,White Non Hispanic,65,Yates,1357
11696,2017,7,45 to 54,2,Female,1,White Non Hispanic,65,Yates,1409
11697,2017,8,55 to 64,1,Male,1,White Non Hispanic,65,Yates,1767
11698,2017,4,20 to 24,1,Male,1,White Non Hispanic,65,Yates,781


Now we will make our dataset so that it doesn't include rows that contain `Total` in them... for each of the three columns.

In [4]:
df_17 = data_17[~data_17['Age Group Description'].isin(['Total'])]
#df_17
df_17.shape

(10725, 10)

In [5]:
df_17['Age Group Description'].unique()

array(['65 to 74', '45 to 54', '35 to 44', '25 to 34', '20 to 24',
       '10 to 19', '1 to 9', '85+', '75 to 84', 'Less than 1', '55 to 64'],
      dtype=object)

In [6]:
df_17['Race/Ethnicity Description'].unique()

array(['Other Non Hispanic', 'White Non Hispanic', 'Total',
       'Black Non Hispanic', 'Hispanic'], dtype=object)

In [7]:
df_17 = df_17[~df_17['Race/Ethnicity Description'].isin(['Total'])]
#df_17
df_17.shape

(8580, 10)

In [8]:
df_17['Race/Ethnicity Description'].unique()

array(['Other Non Hispanic', 'White Non Hispanic', 'Black Non Hispanic',
       'Hispanic'], dtype=object)

In [9]:
df_17['Gender Description'].unique()

array(['Female', 'Male', 'Total'], dtype=object)

In [10]:
df_17 = df_17[~df_17['Gender Description'].isin(['Total'])]
#df_17
df_17.shape

(5720, 10)

In [11]:
df_17['Gender Description'].unique()

array(['Female', 'Male'], dtype=object)

Now we can start splitting the population data by gender. Our values are sorted by `County` (alphabetical order) - and we should probably check to make sure we have the correct number of counties (62).

In [12]:
df_17['County Name'].nunique()

65

It appears we have some extra county labels - we will investigate further.

In [13]:
df_17['County Name'].unique()

array(['Albany', 'Allegany', 'Bronx', 'Broome', 'Cattaraugus', 'Cayuga',
       'Chautauqua', 'Chemung', 'Chenango', 'Clinton', 'Columbia',
       'Cortland', 'Delaware', 'Dutchess', 'Erie', 'Essex', 'Franklin',
       'Fulton', 'Genesee', 'Greene', 'Hamilton', 'Herkimer', 'Jefferson',
       'Kings', 'Lewis', 'Livingston', 'Madison', 'Monroe', 'Montgomery',
       'Nassau', 'New York', 'New York City', 'New York State', 'Niagara',
       'Oneida', 'Onondaga', 'Ontario', 'Orange', 'Orleans', 'Oswego',
       'Otsego', 'Putnam', 'Queens', 'Rensselaer', 'Rest of State',
       'Richmond', 'Rockland', 'Saratoga', 'Schenectady', 'Schoharie',
       'Schuyler', 'Seneca', 'St Lawrence', 'Steuben', 'Suffolk',
       'Sullivan', 'Tioga', 'Tompkins', 'Ulster', 'Warren', 'Washington',
       'Wayne', 'Westchester', 'Wyoming', 'Yates'], dtype=object)

These are the extra `County` labels: `New York` (location: 30th element in the above array), `New York State` (loc: 32), `Rest of State` (loc: 44). We will remove them shortly. 

We have Male and Female labels under the same column `Gender Description`, as well as `Gender Code` (2 = Female, 1 = Male). We want to have **two new columns**: `Female Population` and `Male Population`. 

To do this, we need to split this subset into one containing only Female data, and the other containing only Male data. 

In [14]:
df_17.to_csv(path+'gender_17.csv') # saving so I can easily just include the columns
# I want, now that I got rid of the unwanted "Total" rows.

In [15]:
cols = ['Gender Description', 'County Name', 'Population']
df_17 = pd.read_csv(path+'gender_17.csv', usecols=cols)
df_17

,Gender Description,County Name,Population
0,Female,Albany,491
1,Male,Albany,14526
2,Female,Albany,12018
3,Male,Albany,12066
4,Female,Albany,13525
...,...,...,...
5715,Female,Yates,1132
5716,Male,Yates,1357
5717,Female,Yates,1409
5718,Male,Yates,1767


In [16]:
female = df_17.loc[(df_17['Gender Description']=="Female")].reset_index(drop=True)
female

,Gender Description,County Name,Population
0,Female,Albany,491
1,Female,Albany,12018
2,Female,Albany,13525
3,Female,Albany,11201
4,Female,Albany,12535
...,...,...,...
2855,Female,Yates,1541
2856,Female,Yates,933
2857,Female,Yates,1252
2858,Female,Yates,1132


Now population in this subset can be seen as `Female Population`! Let's rename this row for clarity (and future use).

In [17]:
# Population represents number of females here!
fpop_17 = female.rename(columns={'Population':'Female Population'})
fpop_17

,Gender Description,County Name,Female Population
0,Female,Albany,491
1,Female,Albany,12018
2,Female,Albany,13525
3,Female,Albany,11201
4,Female,Albany,12535
...,...,...,...
2855,Female,Yates,1541
2856,Female,Yates,933
2857,Female,Yates,1252
2858,Female,Yates,1132


Now we will get the total female population for each county.

In [18]:
fpop_17 = fpop_17.groupby('County Name').sum().reset_index()
fpop_17

,County Name,Female Population
0,Albany,159866
1,Allegany,23042
2,Bronx,777359
3,Broome,98358
4,Cattaraugus,38897
...,...,...
60,Washington,29727
61,Wayne,45691
62,Westchester,505305
63,Wyoming,18504


In [19]:
fpop_17['County Name'].unique()

array(['Albany', 'Allegany', 'Bronx', 'Broome', 'Cattaraugus', 'Cayuga',
       'Chautauqua', 'Chemung', 'Chenango', 'Clinton', 'Columbia',
       'Cortland', 'Delaware', 'Dutchess', 'Erie', 'Essex', 'Franklin',
       'Fulton', 'Genesee', 'Greene', 'Hamilton', 'Herkimer', 'Jefferson',
       'Kings', 'Lewis', 'Livingston', 'Madison', 'Monroe', 'Montgomery',
       'Nassau', 'New York', 'New York City', 'New York State', 'Niagara',
       'Oneida', 'Onondaga', 'Ontario', 'Orange', 'Orleans', 'Oswego',
       'Otsego', 'Putnam', 'Queens', 'Rensselaer', 'Rest of State',
       'Richmond', 'Rockland', 'Saratoga', 'Schenectady', 'Schoharie',
       'Schuyler', 'Seneca', 'St Lawrence', 'Steuben', 'Suffolk',
       'Sullivan', 'Tioga', 'Tompkins', 'Ulster', 'Warren', 'Washington',
       'Wayne', 'Westchester', 'Wyoming', 'Yates'], dtype=object)

Now we will remove the extra county categories. Recall the locations of these "Counties":

* New York = 30
* New York State = 32
* Rest of State = 44

**However, after we drop New York - their locations will shift up (or closer to 1). So actually the locations will follow this logic:**
1. Drop `New York` at location `30`. That means `New York City` will take over that location, meaning `New York State` will shift to `31`.
2. Drop `New York State` at location `31`.
3. `Rest of State` should be dropped at `42`, because it will have shifted to `43` in step 1, and `42` at step 2.

In [20]:
fpop_17 = fpop_17.drop(30, axis=0).reset_index(drop=True)
fpop_17 = fpop_17.drop(31, axis=0).reset_index(drop=True)
fpop_17 = fpop_17.drop(42, axis=0).reset_index(drop=True)
fpop_17['County Name'].unique()

array(['Albany', 'Allegany', 'Bronx', 'Broome', 'Cattaraugus', 'Cayuga',
       'Chautauqua', 'Chemung', 'Chenango', 'Clinton', 'Columbia',
       'Cortland', 'Delaware', 'Dutchess', 'Erie', 'Essex', 'Franklin',
       'Fulton', 'Genesee', 'Greene', 'Hamilton', 'Herkimer', 'Jefferson',
       'Kings', 'Lewis', 'Livingston', 'Madison', 'Monroe', 'Montgomery',
       'Nassau', 'New York City', 'Niagara', 'Oneida', 'Onondaga',
       'Ontario', 'Orange', 'Orleans', 'Oswego', 'Otsego', 'Putnam',
       'Queens', 'Rensselaer', 'Richmond', 'Rockland', 'Saratoga',
       'Schenectady', 'Schoharie', 'Schuyler', 'Seneca', 'St Lawrence',
       'Steuben', 'Suffolk', 'Sullivan', 'Tioga', 'Tompkins', 'Ulster',
       'Warren', 'Washington', 'Wayne', 'Westchester', 'Wyoming', 'Yates'],
      dtype=object)

In [21]:
fpop_17

,County Name,Female Population
0,Albany,159866
1,Allegany,23042
2,Bronx,777359
3,Broome,98358
4,Cattaraugus,38897
...,...,...
57,Washington,29727
58,Wayne,45691
59,Westchester,505305
60,Wyoming,18504


Looks good!

In [22]:
fpop_17.isna().sum()

County Name          0
Female Population    0
dtype: int64

Now we do the same for male data.

In [23]:
male = df_17.loc[(df_17['Gender Description']=="Male")].reset_index(drop=True)
male

,Gender Description,County Name,Population
0,Male,Albany,14526
1,Male,Albany,12066
2,Male,Albany,14097
3,Male,Albany,11333
4,Male,Albany,12696
...,...,...,...
2855,Male,Yates,1277
2856,Male,Yates,1088
2857,Male,Yates,1357
2858,Male,Yates,1767


In [24]:
# Population represents number of males here!
mpop_17 = male.rename(columns={'Population':'Male Population'})
mpop_17

,Gender Description,County Name,Male Population
0,Male,Albany,14526
1,Male,Albany,12066
2,Male,Albany,14097
3,Male,Albany,11333
4,Male,Albany,12696
...,...,...,...
2855,Male,Yates,1277
2856,Male,Yates,1088
2857,Male,Yates,1357
2858,Male,Yates,1767


In [25]:
mpop_17 = mpop_17.groupby('County Name').sum().reset_index()
mpop_17

,County Name,Male Population
0,Albany,149746
1,Allegany,23852
2,Bronx,693801
3,Broome,95281
4,Cattaraugus,38451
...,...,...
60,Washington,31893
61,Wayne,44979
62,Westchester,474939
63,Wyoming,21989


In [26]:
mpop_17['County Name'].unique()

array(['Albany', 'Allegany', 'Bronx', 'Broome', 'Cattaraugus', 'Cayuga',
       'Chautauqua', 'Chemung', 'Chenango', 'Clinton', 'Columbia',
       'Cortland', 'Delaware', 'Dutchess', 'Erie', 'Essex', 'Franklin',
       'Fulton', 'Genesee', 'Greene', 'Hamilton', 'Herkimer', 'Jefferson',
       'Kings', 'Lewis', 'Livingston', 'Madison', 'Monroe', 'Montgomery',
       'Nassau', 'New York', 'New York City', 'New York State', 'Niagara',
       'Oneida', 'Onondaga', 'Ontario', 'Orange', 'Orleans', 'Oswego',
       'Otsego', 'Putnam', 'Queens', 'Rensselaer', 'Rest of State',
       'Richmond', 'Rockland', 'Saratoga', 'Schenectady', 'Schoharie',
       'Schuyler', 'Seneca', 'St Lawrence', 'Steuben', 'Suffolk',
       'Sullivan', 'Tioga', 'Tompkins', 'Ulster', 'Warren', 'Washington',
       'Wayne', 'Westchester', 'Wyoming', 'Yates'], dtype=object)

In [27]:
mpop_17 = mpop_17.drop(30, axis=0).reset_index(drop=True)
mpop_17 = mpop_17.drop(31, axis=0).reset_index(drop=True)
mpop_17 = mpop_17.drop(42, axis=0).reset_index(drop=True)
mpop_17['County Name'].unique()

array(['Albany', 'Allegany', 'Bronx', 'Broome', 'Cattaraugus', 'Cayuga',
       'Chautauqua', 'Chemung', 'Chenango', 'Clinton', 'Columbia',
       'Cortland', 'Delaware', 'Dutchess', 'Erie', 'Essex', 'Franklin',
       'Fulton', 'Genesee', 'Greene', 'Hamilton', 'Herkimer', 'Jefferson',
       'Kings', 'Lewis', 'Livingston', 'Madison', 'Monroe', 'Montgomery',
       'Nassau', 'New York City', 'Niagara', 'Oneida', 'Onondaga',
       'Ontario', 'Orange', 'Orleans', 'Oswego', 'Otsego', 'Putnam',
       'Queens', 'Rensselaer', 'Richmond', 'Rockland', 'Saratoga',
       'Schenectady', 'Schoharie', 'Schuyler', 'Seneca', 'St Lawrence',
       'Steuben', 'Suffolk', 'Sullivan', 'Tioga', 'Tompkins', 'Ulster',
       'Warren', 'Washington', 'Wayne', 'Westchester', 'Wyoming', 'Yates'],
      dtype=object)

In [28]:
mpop_17

,County Name,Male Population
0,Albany,149746
1,Allegany,23852
2,Bronx,693801
3,Broome,95281
4,Cattaraugus,38451
...,...,...
57,Washington,31893
58,Wayne,44979
59,Westchester,474939
60,Wyoming,21989


In [29]:
mpop_17.isna().sum()

County Name        0
Male Population    0
dtype: int64

Now we have our populations for male and female. We can put them together into a single dataset. It is also important to note that the `Population` data we have in out `flu` subsets will vary slightly, because they were pulled off estimates based on census data. If it becomes a problem, I will redo the `flu_17` subset and put in population data strictly from census data, and re-calculate the prevalence rates. Because we are looking at Female vs. Male population, it should not be an issue for the sake of this project.

In [30]:
# First make total population to make sure data adds up...
df_17

,Gender Description,County Name,Population
0,Female,Albany,491
1,Male,Albany,14526
2,Female,Albany,12018
3,Male,Albany,12066
4,Female,Albany,13525
...,...,...,...
5715,Female,Yates,1132
5716,Male,Yates,1357
5717,Female,Yates,1409
5718,Male,Yates,1767


In [31]:
tpop_17 = df_17.groupby('County Name').sum().reset_index()
tpop_17

,County Name,Population
0,Albany,309612
1,Allegany,46894
2,Bronx,1471160
3,Broome,193639
4,Cattaraugus,77348
...,...,...
60,Washington,61620
61,Wayne,90670
62,Westchester,980244
63,Wyoming,40493


In [32]:
tpop_17 = tpop_17.drop(30, axis=0).reset_index(drop=True)
tpop_17 = tpop_17.drop(31, axis=0).reset_index(drop=True)
tpop_17 = tpop_17.drop(42, axis=0).reset_index(drop=True)
tpop_17['County Name'].unique()

array(['Albany', 'Allegany', 'Bronx', 'Broome', 'Cattaraugus', 'Cayuga',
       'Chautauqua', 'Chemung', 'Chenango', 'Clinton', 'Columbia',
       'Cortland', 'Delaware', 'Dutchess', 'Erie', 'Essex', 'Franklin',
       'Fulton', 'Genesee', 'Greene', 'Hamilton', 'Herkimer', 'Jefferson',
       'Kings', 'Lewis', 'Livingston', 'Madison', 'Monroe', 'Montgomery',
       'Nassau', 'New York City', 'Niagara', 'Oneida', 'Onondaga',
       'Ontario', 'Orange', 'Orleans', 'Oswego', 'Otsego', 'Putnam',
       'Queens', 'Rensselaer', 'Richmond', 'Rockland', 'Saratoga',
       'Schenectady', 'Schoharie', 'Schuyler', 'Seneca', 'St Lawrence',
       'Steuben', 'Suffolk', 'Sullivan', 'Tioga', 'Tompkins', 'Ulster',
       'Warren', 'Washington', 'Wayne', 'Westchester', 'Wyoming', 'Yates'],
      dtype=object)

In [33]:
tpop_17

,County Name,Population
0,Albany,309612
1,Allegany,46894
2,Bronx,1471160
3,Broome,193639
4,Cattaraugus,77348
...,...,...
57,Washington,61620
58,Wayne,90670
59,Westchester,980244
60,Wyoming,40493


In [34]:
tpop_17.isna().sum()

County Name    0
Population     0
dtype: int64

In [35]:
m = mpop_17['Male Population']
t = tpop_17['Population']
pop_17 = fpop_17
pop_17['Male Population'] = m
pop_17['Total Population'] = t
pop_17

,County Name,Female Population,Male Population,Total Population
0,Albany,159866,149746,309612
1,Allegany,23042,23852,46894
2,Bronx,777359,693801,1471160
3,Broome,98358,95281,193639
4,Cattaraugus,38897,38451,77348
...,...,...,...,...
57,Washington,29727,31893,61620
58,Wayne,45691,44979,90670
59,Westchester,505305,474939,980244
60,Wyoming,18504,21989,40493


In [36]:
#pop_17.to_csv(path+"pop_17")

Now we have the population data and it appears to add up. I saved it as a csv so we can use it for the next part (where we will add it to the `flu_17` subset). I am going to use the population data from the census data (above), but will leave the prevalence rates alone (if I have time, I will recalculate it, but it truly should not make a huge difference for the scope of this project)

In [37]:
cols = ['County','Count','Influenza_A','Influenza_B','Influenza_Unspecified','FIPS','Prevalence_Rate']
flu_17 = pd.read_csv(path+"flu_17.csv", usecols=cols)

flu_17 = flu_17.rename(columns={'Influenza_A':'Influenza A',
                               'Influenza_B':'Influenza B',
                               'Influenza_Unspecified':'Influenza Unspecified',
                               'Prevalence_Rate':'Prevalence Rate'})
flu_17

,County,Count,Influenza A,Influenza B,Influenza Unspecified,FIPS,Prevalence Rate
0,ALBANY,1708,1234,456,18,36001.0,56.074275
1,ALLEGANY,205,141,58,6,36003.0,42.008197
2,BRONX,11749,6784,4740,225,36005.0,84.081484
3,BROOME,2214,1618,584,12,36007.0,111.053706
4,CATTARAUGUS,492,235,255,2,36009.0,61.642548
...,...,...,...,...,...,...,...
57,WASHINGTON,278,191,86,1,36115.0,44.063337
58,WAYNE,1413,906,505,2,36117.0,151.518401
59,WESTCHESTER,8639,4767,3841,31,36119.0,90.341350
60,WYOMING,284,167,117,0,36121.0,67.863031


In [38]:
f = fpop_17['Female Population']
flu_17['Female Population'] = f
flu_17['Male Population'] = m
flu_17['Total Population'] = t
flu_17

,County,Count,Influenza A,Influenza B,Influenza Unspecified,FIPS,Prevalence Rate,Female Population,Male Population,Total Population
0,ALBANY,1708,1234,456,18,36001.0,56.074275,159866,149746,309612
1,ALLEGANY,205,141,58,6,36003.0,42.008197,23042,23852,46894
2,BRONX,11749,6784,4740,225,36005.0,84.081484,777359,693801,1471160
3,BROOME,2214,1618,584,12,36007.0,111.053706,98358,95281,193639
4,CATTARAUGUS,492,235,255,2,36009.0,61.642548,38897,38451,77348
...,...,...,...,...,...,...,...,...,...,...
57,WASHINGTON,278,191,86,1,36115.0,44.063337,29727,31893,61620
58,WAYNE,1413,906,505,2,36117.0,151.518401,45691,44979,90670
59,WESTCHESTER,8639,4767,3841,31,36119.0,90.341350,505305,474939,980244
60,WYOMING,284,167,117,0,36121.0,67.863031,18504,21989,40493


In [40]:
#flu_17.to_csv(path+'data_17.csv')